In [36]:
import pandas as pd
from nltk.probability import FreqDist
from nltk.tokenize import word_tokenize
import math
from operator import itemgetter
import gc

In [37]:
# Read the CSV file into a DataFrame
path = 'C:\\Users\\ritaf\\Documents\\ESTUDOS\\NOVA IMS\\3º Ano\\Text Mining\\Projeto\\new_data\\'
data = pd.read_csv(path + 'lyrics_fdist_lem_stem_2500.csv')

In [38]:
data = data[['lyrics_string_fdist', 'tag']]

In [39]:
# Create a dictionary to store data for each genre
genre_freqs = {}

# Create a list to store all tokens from all genres
all_tokens = []

# Iterate through unique genres
for genre in data['tag'].unique():
    # Create a subset of the DataFrame for the current genre
    genre_df = data.loc[data['tag'] == genre].drop(columns=['tag'])
    
    # Drop NaN values in 'lyrics_string_fdist' ISTO VAI TER DE SE TIRAR
    genre_df.dropna(subset=['lyrics_string_fdist'], inplace=True)
    
    # Join the 'lyrics_string_fdist' column and tokenize
    genre_lyrics = ' '.join(list(genre_df['lyrics_string_fdist']))
    genre_tokens = word_tokenize(genre_lyrics)
    
    # Append tokens to the list for overall frequency distribution
    all_tokens.extend(genre_tokens)

    # Calculate frequency distribution for the current genre
    genre_freq = FreqDist(genre_tokens)
    
    # Store the data in the dictionary
    genre_freqs[f'freq_{genre}'] = genre_freq

# Calculate overall frequency distribution
overall_freq = FreqDist(all_tokens)

# Add the overall frequency distribution to the dictionary
genre_freqs['freq_all'] = overall_freq

# Access the frequency distributions for each genre and the overall frequency using genre_data
# For example: genre_data['freq_pop'], genre_data['freq_rap'], genre_data['overall_freq'], etc.


In [40]:
# ESTA PODE TER PALAVRAS REPETIDAS


# def log_ratio(genre_freqs, n=20):
#     # Get the overall frequency distribution for all genres
#     all_freq = genre_freqs['freq_all']

#     # Create a dictionary to store the log ratios for each genre
#     genre_log_ratios = {}

#     for genre, genre_freq in genre_freqs.items():
#         # Skip the overall frequency distribution
#         if genre == 'freq_all':
#             continue

#         # Calculate total number of words in genre and in all genres
#         total_genre_words = genre_freq.N()
#         total_all_words = all_freq.N()

#         # Calculate the genre's 100 top words
#         genre_freq_100 = genre_freq.most_common(100)

#         # Calculate log ratios for the 100 top words
#         log_ratios = {
#             word: math.log(((freq + 1) / (total_genre_words + 1)) / ((all_freq[word] + 1) / (total_all_words + 1)))
#             for word, freq in genre_freq_100
#         }

#         # Sort log ratios
#         sorted_log_ratios = sorted(log_ratios.items(), key=itemgetter(1), reverse=True)

#         # Store the top log ratios for the genre
#         genre_log_ratios[genre] = sorted_log_ratios[:n]

#     return genre_log_ratios

# # Assuming genre_freqs is a dictionary with frequency distributions for each genre
# # (including an entry with key 'overall_freq' for the overall frequency distribution)
# # Example usage:
# top_words = log_ratio(genre_freqs)

# # Print top words for each genre
# for genre, words in top_words.items():
#     print(f"Top words for {genre}: {words}")


In [41]:
# ESTA GARANTE QUE AS PALAVRAS SÃO ÚNICAS

def log_ratio(genre_freqs, n=20):
    # Get the overall frequency distribution for all genres
    all_freq = genre_freqs['freq_all']

    # Create a dictionary to store the log ratios for each genre
    genre_log_ratios = {}

    # Create a set to keep track of selected words across genres
    selected_words_set = set()

    total_all_words = all_freq.N()

    for genre, genre_freq in genre_freqs.items():
        # Skip the overall frequency distribution
        if genre == 'freq_all':
            continue

        # Calculate total number of words in genre and in all genres
        total_genre_words = genre_freq.N()

        # Calculate the genre's top words
        genre_freq_top = genre_freq.most_common(n*5)

        # Calculate log ratios for the top words
        log_ratios = {
            word: math.log(((freq + 1) / (total_genre_words + 1)) / ((all_freq[word] + 1) / (total_all_words + 1)))
            for word, freq in genre_freq_top
        }

        # Sort log ratios
        sorted_log_ratios = sorted(log_ratios.items(), key=itemgetter(1), reverse=True)

        # Select the top words that are not already selected
        selected_words = []
        for word, ratio in sorted_log_ratios:
            if word not in selected_words_set:
                selected_words.append((word, ratio))
                selected_words_set.add(word)

                # Break once n unique words are selected for the genre
                if len(selected_words) == n:
                    break

        # Store the top log ratios for the genre
        genre_log_ratios[genre] = selected_words

    return genre_log_ratios


In [56]:
# Finding the top words
top_words = log_ratio(genre_freqs, n=100)

# Taking a look at the top words for each genre
for genre, words in top_words.items():
    print(f"Top words for {genre}: {words}")

Top words for freq_rock: [('solo', 1.0637738859979076), ('instrumental', 0.999106831494466), ('flesh', 0.9661237862534917), ('shadows', 0.9278887355385051), ('bones', 0.9182438736698911), ('darkness', 0.8936300339633755), ('burn', 0.8548411286382759), ('fate', 0.8304410734470147), ('burning', 0.82825537656492), ('souls', 0.78663149789252), ('death', 0.7760902149204332), ('blood', 0.7661558987959288), ('escape', 0.7326541281772472), ('lies', 0.7308767238521007), ('beyond', 0.7189267081021335), ('path', 0.7093771883254596), ('fear', 0.7050072864982663), ('fade', 0.6912848519086802), ('walls', 0.6821116669332967), ('scream', 0.6818233559694457), ('nowhere', 0.6749083161992776), ('empty', 0.6736477333358536), ('rise', 0.6721993939585033), ('dying', 0.660178956433014), ('hole', 0.6586693533073917), ('refrain', 0.6432853987801768), ('broken', 0.6405339112741715), ('breath', 0.640386949030479), ('alive', 0.6223168050899422), ('grave', 0.6198080037011078), ('dead', 0.6152011021884276), ('blind

In [57]:
words_only = [word for sublist in top_words.values() for word, _ in sublist]
unique_word_count = len(set(words_only))
print("Number of unique words:", unique_word_count)
print(words_only)

Number of unique words: 600
['solo', 'instrumental', 'flesh', 'shadows', 'bones', 'darkness', 'burn', 'fate', 'burning', 'souls', 'death', 'blood', 'escape', 'lies', 'beyond', 'path', 'fear', 'fade', 'walls', 'scream', 'nowhere', 'empty', 'rise', 'dying', 'hole', 'refrain', 'broken', 'breath', 'alive', 'grave', 'dead', 'blind', 'silence', 'anymore', 'storm', 'inside', 'skin', 'lives', 'become', 'prechorus', 'within', 'away', 'apart', 'sea', 'end', 'sun', 'carry', 'ground', 'fire', 'evil', 'bridge', 'hide', 'dark', 'save', 'stone', 'earth', 'breathe', 'tear', 'war', 'behind', 'fight', 'memories', 'eyes', 'sky', 'everyone', 'falling', 'die', 'knees', 'alone', 'waiting', 'hearts', 'wind', 'anyone', 'dancing', 'theres', 'shame', 'lost', 'light', 'hell', 'waste', 'river', 'blame', 'fall', 'losing', 'across', 'left', 'born', 'comes', 'youll', 'chorus', 'control', 'far', 'nothing', 'ive', 'stand', 'somewhere', 'wall', 'seems', 'keeps', 'pain', 'rappers', 'niggas', 'nigga', 'finna', 'homie', '

In [60]:
data['lyrics_string_fdist'].value_counts()

tell would like lyrics song make find lyrics first help lyrics song would great help                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    

In [63]:
data_lyr = data['lyrics_string_fdist'].dropna()

In [64]:
from sklearn.feature_extraction.text import CountVectorizer
#data_lyr = data.lyrics_string_fdist

def count_vectorizer_to_df(train_lst_strs):

    # Create an instance of the CountVectorizer class - Default vectorizer does not remove stop words
    vectorizer = CountVectorizer(vocabulary=words_only)

    # Fit the vectorizer to the text data and transform the text data into a frequency matrix
    frequency_matrix = vectorizer.fit_transform(train_lst_strs)


    # Convert the frequency matrix to a Pandas DataFrame
    train_df = pd.DataFrame(frequency_matrix.toarray(), columns=vectorizer.get_feature_names_out())

    return train_df

train_df = count_vectorizer_to_df(data_lyr).dropna()
train_df

,solo,instrumental,flesh,shadows,bones,darkness,burn,fate,burning,souls,...,near,strength,child,beauty,strange,power,read,kept,point,sense
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,3,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134952,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
134953,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
134954,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
134955,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
